In [64]:
from utils import *
check_tensorflow_gpu()

proxy = SpatiotemporalCO2()
proxy.__dict__

... Checking Tensorflow Version ...
Tensorflow built with CUDA? True
TF: 2.10.0 | CUDA: 64_112 | cuDNN: 64_8
# GPU available: 1 (NVIDIA GeForce RTX 3080)


{'input_features_dir': 'simulations2D/input_features',
 'output_targets_dir': 'simulations2D/output_targets',
 'n_realizations': 1000,
 'x_channels': 4,
 'y_channels': 2,
 'timesteps': 60,
 'dim': 64,
 'test_size': 0.25,
 't_samples': [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60],
 'optimizer': <keras.optimizers.optimizer_v2.nadam.Nadam at 0x17a6a92ba30>,
 'criterion': <bound method SpatiotemporalCO2.custom_loss of <utils.SpatiotemporalCO2 object at 0x0000017A781D0640>>,
 'L1L2_split': 0.25,
 'loss_alpha': 0.8,
 'regular': <keras.regularizers.L1 at 0x17a6a9294e0>,
 'leaky_slope': 0.25,
 'num_epochs': 100,
 'batch_size': 30,
 'lr_sch_type': 1,
 'lr_decay': 25,
 'verbose': 0}

In [65]:
proxy.load_data()
proxy.process_data(subsample=250)

X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
MinMax Normalization Done!
Subsampling data for 250 samples, 11 timesteps ...
Train - X: (187, 64, 64, 4) | y: (187, 11, 64, 64, 2)
Test  - X: (63, 64, 64, 4) | y: (63, 11, 64, 64, 2)


In [ ]:
model0 = proxy.make_model()
model0.summary()

In [ ]:
model, fit = proxy.train(model0)

In [ ]:
y_train_pred = model.predict(proxy.X_train)
y_test_pred  = model.predict(proxy.X_test)
print('Train pred: {} | Test pred: {}'.format(y_train_pred.shape, y_test_pred.shape))

In [ ]:
plot_results(151, proxy.X_train, proxy.y_train, y_train_pred, proxy.t_samples)

***
# END

In [3]:
x = proxy.X_train[:10]
print(x.shape)

y = proxy.y_train[:10]
print(y.shape)

(10, 64, 64, 4)
(10, 11, 64, 64, 2)


In [66]:
def encoder_layer(inp, filt, kern=3, pad='same', slope=0.25):
    _ = SeparableConv2D(filt, kern, padding=pad)(inp)
    _ = BatchNormalization()(_)
    _ = LeakyReLU(slope)(_)
    _ = MaxPooling2D()(_)
    return _

def recurrent_decoder(z_input, residuals, filters=[64, 16, 4], previous_timestep=None):
    def recurrent_step(inp, filt, res, kern=3, pad='same'):
        y = ConvLSTM2D(filt, kern, padding=pad)(inp)
        y = BatchNormalization()(y)
        y = LeakyReLU(0.25)(y)
        y = UpSampling2D()(y)
        y = Concatenate()([y, res])
        y = Conv2D(filt, kern, padding=pad)(y)
        y = Activation('sigmoid')(y)
        y = tf.expand_dims(y,1)
        return y
    def recurrent_last(inp, filt, kern=3, pad='same'):
        y = ConvLSTM2D(filt, kern, padding='same')(inp)
        y = BatchNormalization()(y)
        y = LeakyReLU(0.25)(y)
        y = UpSampling2D()(y)
        y = Conv2D(proxy.y_channels, kern, padding='same')(y)
        y = Activation('sigmoid')(y)
        y = tf.expand_dims(y, 1)
        return y
    res3, res2 = residuals
    filt3, filt2, filt1 = filters
    _ = tf.expand_dims(z_input, 1)
    _ = recurrent_step(_, filt3, res3)
    _ = recurrent_step(_, filt2, res2)
    _ = recurrent_last(_, filt1)
    if previous_timestep != None:
        _ = Concatenate(axis=1)([previous_timestep, _])
    return _

def make_model():
    K.clear_session()

    inp = Input(x.shape[1:])
    z1 = encoder_layer(inp,16)
    z2 = encoder_layer(z1, 64)
    z3 = encoder_layer(z2, 128)
    encoder = Model(inp, z3, name='encoder')

    t0  = recurrent_decoder(z3, [z2,z1])
    t1  = recurrent_decoder(z3, [z2,z1], previous_timestep=t0)
    t2  = recurrent_decoder(z3, [z2,z1], previous_timestep=t1)
    t3  = recurrent_decoder(z3, [z2,z1], previous_timestep=t2)
    t4  = recurrent_decoder(z3, [z2,z1], previous_timestep=t3)
    t5  = recurrent_decoder(z3, [z2,z1], previous_timestep=t4)
    t6  = recurrent_decoder(z3, [z2,z1], previous_timestep=t5)
    t7  = recurrent_decoder(z3, [z2,z1], previous_timestep=t6)
    t8  = recurrent_decoder(z3, [z2,z1], previous_timestep=t7)
    t9  = recurrent_decoder(z3, [z2,z1], previous_timestep=t8)
    t10 = recurrent_decoder(z3, [z2,z1], previous_timestep=t9)
    model = Model(inp, t10, name='CNN_RNN_Proxy')

    return model, encoder


In [67]:
model, encoder = make_model()
model.summary()

Model: "CNN_RNN_Proxy"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 4)]  0           []                               
                                                                                                  
 separable_conv2d (SeparableCon  (None, 64, 64, 16)  116         ['input_1[0][0]']                
 v2D)                                                                                             
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 16)  64          ['separable_conv2d[0][0]']       
 alization)                                                                                       
                                                                                      

In [68]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
fit = model.fit(proxy.X_train, proxy.y_train, 
                epochs = 10,
                batch_size = 15,
                validation_split = 0.2,
                shuffle = True,
                verbose = 1)

Epoch 1/10
